In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Wine-Quality-Prediction'

In [4]:
from mlproject.constants import *

[2025-02-20 23:01:42,932 : INFO : __init__ : Logger has been set up successfully!]


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    subsample: float
    num_leaves: int
    n_estimators: int
    max_depth: int
    learning_rate: float
    lambda_l2: float
    lambda_l1: float
    colsample_bytree: float

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories
from mlproject import logger

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LGBMClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            subsample = params.subsample,
            num_leaves = params.num_leaves,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            learning_rate = params.learning_rate,
            lambda_l2 = params.lambda_l2,
            lambda_l1 = params.lambda_l1,
            colsample_bytree = params.colsample_bytree
        )

        return model_trainer_config

In [8]:
import pandas as pd
import numpy as np
import joblib
from lightgbm  import LGBMClassifier

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Validate file paths
        if not os.path.exists(self.config.train_data_path):
            raise FileNotFoundError(f"Training data file not found at {self.config.train_data_path}")
        if not os.path.exists(self.config.test_data_path):
            raise FileNotFoundError(f"Testing data file not found at {self.config.test_data_path}")

        # Load the data
        train_data = np.load(self.config.train_data_path, allow_pickle=True)
        test_data = np.load(self.config.test_data_path, allow_pickle=True)

        logger.info(f"Loaded train data: type={type(train_data)}, shape={train_data.shape}")
        logger.info(f"Loaded test data: type={type(test_data)}, shape={test_data.shape}")

        # Split features and target
        train_x = train_data[:, :-1]  # All columns except the last one
        train_y = train_data[:, -1]   # Only the last column
        test_x = test_data[:, :-1]    # All columns except the last one
        test_y = test_data[:, -1]     # Only the last column

        logger.info(f"Training data shape: X={train_x.shape}, y={train_y.shape}")
        logger.info(f"Testing data shape: X={test_x.shape}, y={test_y.shape}")

        # Train the model
        logger.info("Initializing RandomForestClassifier...")
        classifier = LGBMClassifier(n_estimators=self.config.n_estimators,
                                            max_depth=self.config.max_depth,
                                            subsample=self.config.subsample,
                                            num_leaves=self.config.num_leaves,
                                            learning_rate=self.config.learning_rate,
                                            lambda_l2=self.config.lambda_l2,
                                            lambda_l1=self.config.lambda_l1,
                                            colsample_bytree=self.config.colsample_bytree,
                                            random_state=42,verbose=-1)
        classifier.fit(train_x, train_y)

        logger.info("Training the model...")

        # Save the trained model
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(classifier, model_path)
        logger.info(f"Model saved successfully at {model_path}")

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-02-20 23:01:45,971 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-02-20 23:01:45,972 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-02-20 23:01:45,981 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-02-20 23:01:45,982 : INFO : common : created directory at: artifacts]
[2025-02-20 23:01:45,982 : INFO : common : created directory at: artifacts/model_trainer]
[2025-02-20 23:01:46,021 : INFO : 1750983693 : Loaded train data: type=<class 'numpy.ndarray'>, shape=(7761, 20)]
[2025-02-20 23:01:46,021 : INFO : 1750983693 : Loaded test data: type=<class 'numpy.ndarray'>, shape=(2587, 20)]
[2025-02-20 23:01:46,021 : INFO : 1750983693 : Training data shape: X=(7761, 19), y=(7761,)]
[2025-02-20 23:01:46,031 : INFO : 1750983693 : Testing data shape: X=(2587, 19), y=(2587,)]
[2025-02-20 23:01:46,032 : INFO : 1750983693 : Initializing RandomForestClassifier...]
[2025-02-20 23:01:46,396 : INFO : 1750983693 : Training the model..